 # Using declarations

* Variable being declared should be disposed at the end of the enclosing scope
* Allows to avoid unnecessary nesting

```csharp
{
    using var test = new TestDisposable();
    test.DoSomething();
}
```

Same as: 

```csharp
{
    TestDisposable testDisposable = new TestDisposable();
    try
    {
        while (true)
        {
            testDisposable.DoSomething();
        }
    }
    finally
    {
        if (testDisposable != null)
        {
            testDisposable.Dispose();
        }
    }
}
```


In [ ]:
using System.Net.Http;
using System.Text.Json;

public class CatUrl
{
    public string url {get; set;}
}

public class TestDisposable : IDisposable
{
    readonly HttpClient client = new HttpClient();
    public async Task DoSomething()
    {
        HttpResponseMessage response = await client.GetAsync("https://api.thecatapi.com/v1/images/search");
        response.EnsureSuccessStatusCode();
        await response.Content.ReadAsStreamAsync();
        using var responseStream = await response.Content.ReadAsStreamAsync();
        var result = await JsonSerializer.DeserializeAsync<IEnumerable<CatUrl>>(responseStream);
        //"https://www.hdwallpaper.nu/wp-content/uploads/2015/02/Funny-Cat-Hidden.jpg"
        display(img[
            src: result.First().url,
            style: "height: 300px;"]);
    }

    public void Dispose()
    {
        client?.Dispose();
        display("Disposed...");
    }
}

display("Before");

var counter = 3;
{
    using var test = new TestDisposable();
    do
    {
        // TODO: move using declaration here and check
        await test.DoSomething();
    }
    while(--counter > 0);
}


display("After");

## Disposal precedence

```csharp
using var f1 = new TestDisposable("f1");
using TestDisposable f2 = new TestDisposable("f2"), f3 = new TestDisposable("f3");
...
// Dispose f3
// Dispose f2 
// Dispose f1
```

# Demo


In [ ]:
{
    using var f1 = new TestDisposable("f1");
    using TestDisposable f2 = new TestDisposable("f2"), f3 = new TestDisposable("f3");
}

public class TestDisposable : IDisposable
{
    private readonly string _name;
    public TestDisposable(string name) => _name = name;
    public void Dispose()
    {
        display($"Dispose {_name}");
    }
}

## Demo

Explanation: [sharplab-demo-using-declarations](https://sharplab.io/#v2:EYLgtghgzgLgpgJwDQBMQGoA+ABADAAmwEYA6ASQHkBuAWACh7iA2fAWTjAHsEBPAZRgI4EMPgDicGABEIMCAAoAlPQDe9fBsIEAbhAT4wHAUJH4AvPgB2cAO5sO3foOFgltOpq35d+2CbAA6ggAlvD6FtZ2xi5BoYjyhmDRIorunn4xIWEksfAAMsHW8gBEACoAonylxanqmtgA7Fa29ly8ya6JHSSlnACCCAgQPEpI+DZZEMAANnAg+ABmENNQcLV0AL70jEQs7G1O/uKSMnIATEqqdRp43noGRs6mES37jh1u1/hqHp71OvcMiJcohzM0ok9All4l1Ies/pogVC4ggctCCkUypVqvDNFtfjcmpFWu9IQlHv4ev1BsNRuNJjM5otlqt1hsgA==)

In [ ]:
using System.IO;

var stream = GetData();
display(System.Text.Encoding.ASCII.GetString(stream.ToArray()));

public MemoryStream GetData()
{
    // TODO: fix this
    using var memStream = new MemoryStream();
    using var streamWriter = new StreamWriter(memStream);
    streamWriter.WriteLine("TEST");
    return new MemoryStream(memStream.ToArray(), writable: false);
}

// 📝HINT: try to introduce another declaration scope

## Reference

* https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-8#using-declarations
* https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/using
* https://gunnarpeipman.com/csharp-using-declarations/
* https://www.stevejgordon.co.uk/csharp-8-understanding-using-declarations